#### **Instalacja bibliotek**

In [1]:
#tu rzeczy do instalacji, zaczynac od %
#%pip install matplotlib
#%pip install pandas
#tu sklearn, możliwe że trzeba zrestartowac krenel po updacie
#%pip install -U scikit-learn
#%pip install scipy

#### **Importy**

In [2]:
#tu biblioteki
import numpy as np
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print("Import done")

Num GPUs Available:  1
Import done


#### **Przeniesienie obliczeń na GPU**

In [3]:
#GPUs - tu poki co nie dziala, trzeba postawic srodowisko wirtualne
gpus = tf.config.experimental.list_physical_devices('GPU')
print (len(gpus), " GPUs found!")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), " Physical GPUs, ", len(logical_gpus), " Logical GPUs")
    except RuntimeError as e:
        print(e)
print("GPUs done")

1  GPUs found!
1  Physical GPUs,  1  Logical GPUs
GPUs done


#### **Wczytanie obrazków**

In [4]:
def directory_flow(train_directory, val_directory, shape):
    #directory = nazwy folderow,
    #shape = rozmiar obrazka, u nas 256x256
    print ("generators start")
    image_generator=ImageDataGenerator()
    train_gen = image_generator.flow_from_directory(train_directory, target_size=shape, shuffle=True) #sciezka do train dir
    print ("train done")
    val_gen = image_generator.flow_from_directory(val_directory, target_size=shape, shuffle=True)
    print ("validate done")
    print ("generators done")
    #zastanowic sie nad zwracaniem validate gen
    return train_gen, val_gen

In [5]:
#nie zadziala, bo jeszcze nie ma obrazkow
train_gen, val_gen = directory_flow("./archive/TrainingData", "./archive/ValidationData", (256, 256))
print("Directory flow done!")

generators start
Found 30000 images belonging to 15 classes.
train done
Found 2000 images belonging to 15 classes.
validate done
generators done
Directory flow done!


#### **Wstępna analiza danych**

In [6]:
# Do analizy danych
#labels = kategorie (rodzaje zwierzatek)
def plot_label_count(df, plot_title):
    '''
    This function take df and plot labels value counts
    '''

    # Define needed variables
    vcounts = df['labels'].value_counts()
    labels = vcounts.keys().tolist()
    values = vcounts.tolist()
    lcount = len(labels)

    if lcount > 55:
        print('The number of labels is > 55, no plot will be produced')

    else:
        plot_labels(lcount, labels, values, plot_title)

def plot_labels(lcount, labels, values, plot_title):
    width = lcount * 4
    width = np.min([width, 20])

    plt.figure(figsize= (width, 5))

    form = {'family': 'serif', 'color': 'blue', 'size': 25}
    sns.barplot(x=labels, y=values)
    plt.title(f'Images per Label in {plot_title} data', fontsize= 24, color= 'blue')
    plt.xticks(rotation= 90, fontsize= 18)
    plt.yticks(fontsize= 18)
    plt.xlabel('CLASS', fontdict= form)
    yaxis_label = 'IMAGE COUNT'
    plt.ylabel(yaxis_label, fontdict= form)

    rotation = 'vertical' if lcount >= 8 else 'horizontal'
    for i in range(lcount):
        plt.text(i, values[i] / 2, str(values[i]), fontsize= 12,
                rotation= rotation, color= 'yellow', ha= 'center')

    plt.show()

In [7]:
#Do analizy danych = przykładowe obrazki
def show_images(gen):
    '''
    This function take the data generator and show sample of the images
    '''

    # return classes , images to be displayed
    g_dict = gen.class_indices        # defines dictionary {'class': index}
    classes = list(g_dict.keys())     # defines list of dictionary's kays (classes), classes names : string
    images, labels = next(gen)        # get a batch size samples from the generator

    # calculate number of displayed samples
    length = len(labels)        # length of batch size
    sample = min(length, 25)    # check if sample less than 25 images

    plt.figure(figsize= (20, 20))

    for i in range(sample):
        plt.subplot(5, 5, i + 1)
        image = images[i] / 255       # scales data to range (0 - 255)
        plt.imshow(image)
        index = np.argmax(labels[i])  # get image index
        class_name = classes[index]   # get class of image
        plt.title(class_name, color= 'blue', fontsize= 12)
        plt.axis('off')
    plt.show()

In [8]:
#analiza danych - zadziala jak zamiast train_gen i val_gen da się dataframy
#pokaz ile obrazkow w kategorii

#plot_label_count(train_gen, 'train')
#plot_label_count(val_gen, 'validate')

#### **Modele**

In [9]:
#model madry - xception trenowany wstepnie(weights = imagenet)
def make_smart_xception_pretrained_model(shape):
    #te rzeczy w nawiasie zapobiegna przetrenowaniu
    base_model = tf.keras.applications.xception.Xception(include_top=False, weights="imagenet",input_shape=shape, pooling='max')
    model = Sequential([
        base_model,
        Dense(15, activation= 'softmax')
        ])
    #ograniczenie warstw zapobiegnie przetrenowaniu
    model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return model
print("Model done!")

Model done!


In [10]:
#model glupi - tylko na xception, nie trenowany wstepnie bo weights=none
def make_dumb_xception_non_pretrained_model(shape):
    base_model = tf.keras.applications.xception.Xception(include_top=True, weights=None,input_shape=shape, pooling='max', classes=15)
    base_model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return base_model
print("Model done!")

Model done!


In [11]:
#model glupi - taki jak na zajeciach, ale najwiecej warstw
def make_dumb_the_biggest_classic_model(shape):
    model = Sequential()
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu", input_shape = shape))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Flatten())
    model.add(Dense(15, activation="softmax"))
    model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return model
print("Model done!")

Model done!


In [12]:
#model glupi - taki jak na zajeciach, ale wiecej warstw
def make_dumb_bigger_classic_model(shape):
    model = Sequential()
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu", input_shape = shape))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Flatten())
    model.add(Dense(15, activation="softmax"))
    model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return model
print("Model done!")


Model done!


In [13]:
#model glupi - taki jak na zajeciach
def make_dumb_classic_model(shape):
    model = Sequential()
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu", input_shape = shape))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Conv2D(kernel_size=(3,3), filters = 3, activation="relu"))
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dense(15, activation="softmax"))
    model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return model
print("Model done!")

Model done!


#### **Utworzenie modelu i rozpoczęcie treningu**

In [14]:
#model = make_dumb_classic_model((256, 256, 3))
#model = make_dumb_bigger_classic_model((256, 256, 3))
#model = make_dumb_the_biggest_classic_model((256, 256, 3))
#model = make_dumb_xception_non_pretrained_model((256,256,3))
model = make_smart_xception_pretrained_model((256, 256, 3))

print("Model generated")

Model generated


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dense (Dense)               (None, 15)                30735     
                                                                 
Total params: 20,892,215
Trainable params: 20,837,687
Non-trainable params: 54,528
_________________________________________________________________


In [16]:
#zadziala jak wrzuce obrazki
model.fit(x=train_gen, epochs=10, validation_data=(val_gen),validation_steps= None, shuffle=True)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/xception/block2_sepconv1_bn/FusedBatchNormV3' defined at (most recent call last):
    File "c:\Users\Zuza\.conda\envs\tensorflow-g\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Zuza\.conda\envs\tensorflow-g\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Zuza\.conda\envs\tensorflow-g\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Zuza\.conda\envs\tensorflow-g\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\Zuza\.conda\envs\tensorflow-g\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\Users\Zuza\.conda\envs\tensorflow-g\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forev